In [1]:
import os
from collections import defaultdict
from itertools import product
from os import listdir
from os.path import isdir, join

import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
methods = [
    "fixmatch-ours",
    "freematch-ours",
]
datasets = ["cifar100"]
seeds = [0, 1, 2]
schedule = "schedule-2"
names = list(
    map(
        lambda x: "P--" + x[0] + "_" + x[1] + "_" + str(x[2]) + "_" + schedule,
        product(methods, datasets, seeds),
    )
)

results = defaultdict(list)
for name in names:

    path = join(f"{datasets[0]}_{schedule}", name)
    while isdir(path) and "log.txt" not in listdir(path):
        path = join(path, os.listdir(path)[0])
    tb_path = join(path, "tensorboard")
    log_path = join(path, "log.txt")

    with open(log_path, "r") as f:
        lines = f.readlines()

    test_top_1_acc = float(lines[-2].split(":")[-1])
    if "ours" in name:
        method, dataset, seed, _ = name[3:].split("_")
    else:
        method, dataset, seed = name[3:].split("_")

    results["method"].append(method)
    results["dataset"].append(dataset)
    results["seed"].append(seed)
    results["test_top_1_acc"].append(test_top_1_acc * 100)
results_df = pd.DataFrame(results)
results_df.groupby(["method", "dataset"]).test_top_1_acc.agg(["mean", "std"])

,,mean,std
method,dataset,,
fixmatch-ours,cifar100,19.558333,3.066180
freematch-ours,cifar100,24.808333,3.490374
